In [ ]:
import numpy as np
import pandas as pd
import geopandas as gp
import requests
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
from datetime import datetime, timedelta
from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
import shapely.speedups
from shapely import Polygon
import subprocess
from dataclasses import dataclass
import xarray as xr
import cfgrib
import zipfile
import os
import shutil
import rasterio
from rasterio.transform import from_origin
from rasterio.transform import from_bounds
from rasterio.crs import CRS
#import h3
from pyproj import Transformer
import json
import pprint
#import dask
import eccodes
import pygrib
import psycopg2
from osgeo import gdal
from collections import defaultdict
from scipy.spatial import KDTree
import matplotlib.colors as mcolors

@dataclass
class DateTimeParts:
    year: int
    month: int
    day: int
    hour: int
    minute: int

    @classmethod
    def from_datetime(cls, dt: datetime):
        return cls(year=dt.year, month=dt.month, day=dt.day, hour=dt.hour, minute=dt.minute)
    
def windCalc(u,v):
        #print('windCalc Function')
        wind_abs = np.sqrt(u**2 + v**2)
        wind_dir_trig_to = np.arctan2(u/wind_abs, v/wind_abs)
        wind_dir_trig_to_degrees = wind_dir_trig_to * 180/np.pi ## -111.6 degrees
        wind_dir = wind_dir_trig_to_degrees + 180
        return wind_abs * 2.23694 #TO MPH
def K_to_F(temp):
    temp = ((temp - 273.15) * (9/5)) + 32
    return temp

def C_to_F(temp):
    temp = ((temp) * (9/5)) + 32
    return temp

def F_to_K(temp):
    temp = ((temp - 32) * 5/9) + 273.15
    return temp

In [ ]:
url_max ='https://tgftp.nws.noaa.gov/SL.us008001/ST.opnl/DF.gr2/DC.ndfd/AR.conus/VP.001-003/ds.maxt.bin'
url_min ='https://tgftp.nws.noaa.gov/SL.us008001/ST.opnl/DF.gr2/DC.ndfd/AR.conus/VP.001-003/ds.mint.bin'
url_max2 ='https://tgftp.nws.noaa.gov/SL.us008001/ST.opnl/DF.gr2/DC.ndfd/AR.conus/VP.004-007/ds.maxt.bin'
url_min2 ='https://tgftp.nws.noaa.gov/SL.us008001/ST.opnl/DF.gr2/DC.ndfd/AR.conus/VP.004-007/ds.mint.bin'
url_list = [url_max, url_min, url_max2, url_min2]
url_def = ['maxt3', 'mint3', 'maxt7', 'mint7']
result_list = []
for url, def_ in zip(url_list, url_def):
    indexFile = requests.get(url)
    indexFile = (indexFile.content)
    tempfilename = f"data/gribs/ndfd/maxmin/ndfd_{def_}.grb2"
    result_list.append(tempfilename)
    f = open(tempfilename, 'wb')
    f.write(indexFile)
    f.close()

In [ ]:
print(result_list)
model = 'ndfd'
#latestGrb = f'data/gribs/{model.lower()}/latest/{model.lower()}-latest.grb2'
latestGrb = f'data/gribs/{model.lower()}/maxmin/{model.lower()}-latest.grb2'
command_cp = f'cp {result_list[0]} {latestGrb}'
subprocess.call(command_cp, shell=True)

In [ ]:
#latestGrb = result_list[0]
for grib_single in result_list[1:]:
    print(grib_single)
    command_append = f'wgrib2 -append {grib_single} -grib {latestGrb}'
    subprocess.call(command_append, shell=True)

In [ ]:
model = 'ndfd'
#gribfile = f'data/gribs/{model.lower()}/latest/{model.lower()}-024.grb2'
#latestGrb = f'data/gribs/{model.lower()}/latest/{model.lower()}-latest.grb2'
latestGrb = f'data/gribs/{model.lower()}/maxmin/{model.lower()}-latest.grb2'
gribfile = latestGrb
grbs = pygrib.open(gribfile)
for grb in grbs:
    print(grb)

In [ ]:
gribfile = f'data/gribs/{model.lower()}/maxmin/{model.lower()}-240.grb2'
grbs = pygrib.open(gribfile)
try:
    tgrb_max = grbs.select(name = 'Maximum temperature')
except:
    tgrb_max = []
try:
    tgrb_min = grbs.select(name = 'Minimum temperature')
except:
    tgrb_min = []
tempgrbs_max = (tgrb_max)
for  tmp in tempgrbs_max:
    print(tmp)
tempgrbs_min = (tgrb_min)
for  tmp in tempgrbs_min:
    print(tmp)

In [ ]:
#tgrb = grbs.select(shortName = '2t')
tgrb = grbs.select(name = 'Maximum temperature')
#tgrb = grbs.select(paramId = 167) #t2 ens std
tsd = (tgrb[3])
tsd4 = (tgrb[4])
print(tsd)
print(tsd4)
print(dir(tsd))
print(tsd.validDate)
tsd_list = list(dir(tsd))
tsd_keys = (tsd.keys())
print('2 metre temperature:K (instant):lambert:heightAboveGround:level 2 m:fcst time 36 hrs:from 202410151900:ens std dev')
for key in tsd_list:
    try:
        print(key, tsd[key])
    except:
        print(key, ' does not exist')
print('****************************************************')
print('2 metre temperature:K (instant):lambert:heightAboveGround:level 2 m:fcst time 36 hrs:from 202410151900')
tsd4_list = list(dir(tsd4))
tsd4_keys = (tsd4.keys())
for key4 in tsd4_list:
    try:
        print(key4, tsd4[key4])
    except:
        print(key4, ' does not exist')


In [ ]:
grb1 = grbs[1]
print(grb1)
lats, lons = grb1.latlons()
#lons = ((lons + 180) % 360) - 180 #GFS only
# Flatten the lat/lon arrays to create a 2D list of points
grid_points = np.column_stack((lats.ravel(), lons.ravel()))

# Build a KDTree from the lat/lon grid
tree = KDTree(grid_points)

In [ ]:
def createOutput(var, nearest_indices, df_coords):   
    data = {icao: [] for icao in df_coords['ICAO']}
    grbs_all = pygrib.open(gribfile)
    #tgrb = grbs_all.select(shortName = '2t')
    tgrb = grbs.select(name=var)
    tempgrbs = tgrb

    # Loop through the GRIB messages to extract data for all variables and times
    for grb in tempgrbs:
        print(grb)
        var_name = grb.name
        valid_time = grb.validDate

        # Flatten the data grid to align with the grid points
        data_values = (K_to_F(grb.values.ravel()).round(0))

        # For each row in the DataFrame, get the value from the nearest grid point
        for (icao, idx) in zip(df_coords['ICAO'], nearest_indices):
            nearest_value = data_values[idx]

            # Collect the value, time, and variable for the given ICAO
            data[icao].append({
                'time': valid_time,
                'variable': var_name,
                'value': int(nearest_value),
                'lat': df_coords.loc[df_coords['ICAO'] == icao, 'lat'].values[0],
                'lon': df_coords.loc[df_coords['ICAO'] == icao, 'lon'].values[0],
                'city': df_coords.loc[df_coords['ICAO'] == icao, 'CITY'].values[0],  # Add city
                'state': df_coords.loc[df_coords['ICAO'] == icao, 'STATE'].values[0]  # Add state
            })

    # Create an empty list to store rows for the DataFrame
    trows = []

    # Flatten the data dictionary into the list of rows
    for icao, records in data.items():
        for record in records:
            # Append each record as a dictionary to the rows list
            trows.append({
                'ICAO': icao,
                'time': record['time'],
                'variable': record['variable'],
                'value': record['value'],
                'lat': record['lat'],
                'lon': record['lon'],
                'CITY': record['city'],  # Add city to DataFrame
                'STATE': record['state']  # Add state to DataFrame
            })
    print(trows)

    # Convert the list of rows into a DataFrame
    result_df = pd.DataFrame(trows)

    # The final DataFrame contains the time series for each ICAO code
    print(result_df)

    features = []
    for icao, records in data.items():
        # Extract lat/lon from one of the records (all will have the same lat/lon for the same ICAO)
        lat = records[0]['lat']
        lon = records[0]['lon']
        city = records[0]['city']  # Extract city
        state = records[0]['state']  # Extract state

        # Prepare time series for this ICAO
        time_series = [{
            'time': str(record['time']),
            'variable': str(record['variable']),
            'value': str(record['value'])
        } for record in records]

        # Create the GeoJSON feature for this ICAO with the time series
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [str(lon), str(lat)]
            },
            "properties": {
                "ICAO": icao,
                "CITY": city,  # Add city to GeoJSON
                "STATE": state,  # Add state to GeoJSON
                "time_series": time_series
            }
        }
        features.append(feature)

    # Create the final GeoJSON structure
    geojson_data = {
        "type": "FeatureCollection",
        "features": features
    }

    # Determine file name based on the variable type
    if var == 'Maximum temperature':
        var_out = 'maxtemp'
    elif var == 'Minimum temperature':
        var_out = 'mintemp'
    else:
        var_out = ''
    
    # Export to GeoJSON file
    """ with open(f'output_data_{var_out}.geojson', 'w') as f:
        json.dump(geojson_data, f, indent=4) """
    
    # Export to GeoJSON file
    with open(f'/var/www/fapi/app/static/data/point/ndfd/output_data_{var_out}.geojson', 'w') as f:
        json.dump(geojson_data, f, indent=4)


    print("GeoJSON with time series exported successfully!")


df_coords = pd.read_csv('stations.csv')
coordinates = df_coords[['lat', 'lon']].values
nearest_indices = [tree.query([lat, lon])[1] for lat, lon in coordinates]
createOutput('Maximum temperature', nearest_indices, df_coords)
createOutput('Minimum temperature', nearest_indices, df_coords)

In [ ]:
with open('output_data.geojson', 'r') as f:
    geojson_data = json.load(f)

# Print to check structure
#print(json.dumps(geojson_data, indent=4))
print(geojson_data['features'])
print(geojson_data['features'][-1]['properties'])

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('stations2.csv')

# Print the original DataFrame
print("Original DataFrame:")
print(df)

# Define the custom order for the REGION column
regions = ['SOUTH', 'CENTRAL', 'NORTH', 'WEST']

# Convert the 'REGION' column to a categorical type with the specified order
df['REGION'] = pd.Categorical(df['REGION'], categories=regions, ordered=True)

# Sort the DataFrame by 'REGION' (custom order) and 'CITY' (alphabetically)
df_sorted = df.sort_values(['REGION', 'CITY'])

# Print the sorted DataFrame
print("Sorted DataFrame by REGION and CITY:")
print(df_sorted)

In [ ]:
def colortable_gen():
    color_values = {
        -30: 'maroon',
        -20: 'teal',
        -10: 'lavender',
        0: 'white',
        10: 'fuchsia',
        20: 'purple',
        30: 'blue',
        40: 'aqua',
        50: 'green',
        60: 'yellow',
        70: 'orange',
        80: 'red',
        90: 'purple',
        100: 'white',
        110: 'pink',
        130: 'maroon'
    }

    # Create the normalized bounds (0 to 1) for the colormap
    norm = mcolors.Normalize(vmin=min(color_values.keys()), vmax=max(color_values.keys()))

    # Create a LinearSegmentedColormap
    cmap = mcolors.LinearSegmentedColormap.from_list(
        'colormap_tempF',
        [(norm(value), color) for value, color in color_values.items()]
    )

    return cmap

In [ ]:
model = 'ndfd'
latestGrb = f'data/gribs/{model.lower()}/latest/{model.lower()}-latest.grb2'
cmap = colortable_gen()
grbs = pygrib.open(latestGrb)
for grb in grbs[1:2]:

    #prep metadata
    shortname = grb.shortName
    longname = grb.name
    validDate = grb.validDate
    analDate = grb.analDate
    fcstHour = grb.forecastTime
    level = grb.level
    levtype = grb.levtype
    typeOfLevel = grb.typeOfLevel
    unit = grb.units
    cfName = grb.cfVarName
    stepType = grb.stepType
    step = grb.step
    punit = grb.parameterUnits
    print('shortname', shortname)
    print('longname', longname)
    print('stepType', stepType)
    print('step', step)
    #step = timedelta(hours=fcstHour)
    filter_kwargs = {'stepType' : f'{stepType}', 'step': step}
    #filter_kwargs = {'shortName' : f'{shortname}', 'step': step }

    #open grib message as xr dataset using cfgrib engine
    print('opening dataset')
    ds = xr.open_dataset(
        latestGrb,
        engine="cfgrib",
        backend_kwargs={"filter_by_keys": filter_kwargs})
    
    print(ds)
    #ds=ds.isel(y = slice(100,1300), x= slice(800,2100))
    ds = ds.coarsen(latitude=2, longitude=2, boundary='trim').mean()
    ts = ds['unknown']

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
contour = ax.pcolormesh(ds.longitude, ds.latitude, K_to_F(ts.values), transform=ccrs.PlateCarree(), vmin=-30, vmax=130, cmap=cmap, shading='gouraud')
ax.set_extent([-128,-65,23,56], crs=ccrs.PlateCarree())
ax.add_feature(cfeature.STATES)
ax.coastlines()
fig.colorbar(contour, shrink=0.5, orientation='horizontal')

In [ ]:
model = 'ndfd'
latestGrb = f'data/gribs/{model.lower()}/latest/{model.lower()}-latest.grb2'
grbs = pygrib.open(latestGrb)
for grb in grbs:
    print(grb)

In [ ]:
grb = grbs[1]
print(dir(grb))
print(grb.centre)
print(grb.subCentre)